In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Use in case of lack packages 
!pip install -U gensim pyunpack tqdm sklearn

Requirement already up-to-date: gensim in /usr/local/lib/python3.5/dist-packages (3.6.0)
Requirement already up-to-date: pyunpack in /usr/local/lib/python3.5/dist-packages (0.1.2)
Requirement already up-to-date: tqdm in /usr/local/lib/python3.5/dist-packages (4.29.1)
Requirement already up-to-date: sklearn in /usr/local/lib/python3.5/dist-packages (0.0)


In [21]:
from tqdm.autonotebook import tqdm
from wsd import *
from parsers import *

import plwn

In [22]:
remote_embeddings_address = 'http://10.17.5.15:4000'
stop_words_file = 'polish.stopwords.txt'

skladnica = load_skladnica(WSDParseType.wsd)
sw = StopWord(stop_words_file)
wn = Wordnet(plwn.load_default())

we = RemoteWordEmbedding(remote_embeddings_address)
se = RemoteSenseEmbedding(remote_embeddings_address)
# we = get_ft_word_embeddings()
# se = get_sense_embeddings()
wsd = WSD(we, se, sw, wn, tqdm_disable=False)

KeyboardInterrupt: 

In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning, UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)
warnings.filterwarnings(action='ignore', category=RuntimeWarning)
                        
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from functools import partial
from itertools import product, starmap

def write_to_scores(text, filename='scores.csv'):
    with open(filename, 'a+') as f:
        f.write(text + '\n')

def score(idx, sentence, senses):
    skladnica_out = dict(filter(lambda word_sense: word_sense[1], senses))
    wsd_out = wsd(sentence, use_related=False)
    
    def _scores(true, preds):
        acc = accuracy_score(true, preds)
        f1 = f1_score(true, preds, average='macro')
        prec = precision_score(true, preds, average='macro')
        rec = recall_score(true, preds,average='macro')
        return acc, f1, prec, rec
    
    # mati version
    true, preds = [], []
    
    for word, sens_id in wsd_out.items():
        true.append(sens_id[1])
        pred = skladnica_out.get(word)
        preds.append(int(pred) if pred else 0)
    
    mati_vers = _scores(true, preds)

    # grzech version
    true, preds = [], []
    for word, sens_id in skladnica_out.items():
        true.append(int(sens_id))
        pred = wsd_out.get(word)
        preds.append(pred[1] if pred else 0)
    grzech_vers = _scores(true, preds)
    
    write_to_scores(','.join([str(idx), *map(str, mati_vers), *map(str, grzech_vers)]))
    
    return mati_vers, grzech_vers

In [ ]:
write_to_scores('idx,' + ','.join(['_'.join(data) for data in 
                                   product(['mati', 'grzech'], ['acc', 'f1', 'prec', 'rec'])]))
ress = list(starmap(score, tqdm(skladnica, 'Sentence', dynamic_ncols=True)))

In [ ]:
def print_scores(results):
    mati_vers, grzech_vers = np.average(results, axis=0)
    
    def _print_scores(version:str, results):
        print(version, 
              '\t'.join(['{}: {:.2f}',] * 4).format(zip(['acc', 
                                                         'f1', 
                                                         'prec', 
                                                         'rec'], results)))
        
    _print_scores('mati  \t', mati_vers)
    _print_scores('grzech\t', grzech_vers)
    

In [ ]:
print_scores(ress)

In [ ]:
# text = 'zamek królewski'
text = 'zamek w spodniach'
# text = 'zamek w pistolecie'
# text = 'zamek w drzwiach od domu'
# text = 'zamek króla'

wsd_out = wsd(text, use_related=True)

for word, word_out in wsd_out.items():
    lemma, sense = word_out
    print('*', word, ':', get_senses(lemma, wn).get(sense))